In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#script terminologies - The Top Screenwriting Terms for Novice Screenwriters - ScreenCraft
import spacy
import pandas as pd
import nltk
import re
from collections import Iterable
import random

In [0]:
random_list=[]
NUM_WORDS_INFILE=0
def random_gen():                                     #to generate numbers
                                                      #corresponding to rach word, we'll have a random number
    while(True):
        r=random.randint(1,NUM_WORDS_INFILE + 1000)
        if r not in random_list: 
            random_list.append(r)
            break
    return r

filename="gdrive/My Drive/MNF_work/papers/itsamatch.txt"
characters = ['KUSH', 'POOJA', 'DRIVER', 'SHOPKEEPER', 'AKASH', 'RECEPTIONIST']
refined_line=[]

#word count
with open(filename, 'r') as f:   #open file 
    for line in f:
        words = line.split()
        NUM_WORDS_INFILE += len(words)   # count total no. of words
    print("Total number of words in the input file",NUM_WORDS_INFILE)

#read file
with open(filename, "r") as input:
    input_ = input.read().split('\n\n')

for line in input_:
    refined_line.append(line.strip())
print(refined_line)
# whole scenario is read and stored in the refined_line list separated by '\n' wherever a new line is there

words=[]
scenes=[]
scene=[]
priority=[]
word_importance=[]
parenthetical='NONE'
dialogues=[]
actionline=[]
scene_dic = {}
refined_line=list(filter(lambda a: a != "", refined_line))
a_counter=0


##CREATE SCENES LIST ALONG ACTION AND DIALOGUES IN THE GIVEN SEQUENCE   MAY UPDATE THE CODE FROM SHIVAM  ##RENU
for line in refined_line:            #if new scene
    if line.startswith('INT') or line.startswith('EXT') or line.startswith('EXT/INT') or line.startswith('INT/EXT'):
        a_counter=0
        scenes.append(scene)
        print("scene is: {}".format(scene))
        scene=[]
        scene.append(line)
        scene_dic['SL']=line
        continue
    else: #if the line is simple not a scene
                                                       # print("This is Else case and the line is : {}".format(line))
        lis=line.split("\n")                           #take only the first line
        lis=[l.strip() for l in lis]                   # lis is now a list of the particular scwnw separated by a '\n'
        #print("-------------------------------------------------------")
       # print("lis is {}".format(lis))
        word=lis[0]                                    #word is the first line every scene
        #print("word is {}".format(word))
        if word.split('(')[0].strip() in characters:   #if word is a character
            mydic={}
            #spm=re.findall(r'\(([^()]+)\)',word)
            speaker = word.split('(')[0].strip()          #extract the speaker
            if len(lis)>1 and re.match(r"\(.*\)",lis[1]):
                parenthetical=lis[1]
                parenthetical=parenthetical.replace("\n","")   # extract parenthetical
               # print("parenthetical: {}".format(parenthetical))
                dia = " ".join(lis[2:])
                dia = dia.replace("\n","")
                #print("Dialogue: {}".format(dia))
            
            else:
                dia=" ".join(lis[1:])
                dia=dia.replace("\n","")
                #print("Dialogue: {}".format(dia))
            if not (len(dia)==0 and parenthetical=="NONE"):
                mydic[speaker]=[parenthetical,dia,len(dia)]                                                       
                dialogues.append(mydic)                       
                scene.append(mydic)
            parenthetical="NONE"
            #print("dialogue**********\n")
            scene_dic['D']=mydic
            
        else:
            line = line.replace("\n","")
            line =' '.join(line.split())
            actionline.append(line)
            scene.append(line)
            #print("action******\n")
            a_counter= a_counter+1

            scene_dic['A'+str(a_counter)] = line
    #print(scene_dic)
scenes.append(scene)
total_sc_cnt=1
for sc in scenes:
    print(sc)
    total_sc_cnt+=1
print ("total scene count\n", total_sc_cnt)    
print("*************scene data********************\n")

#till now, all the scenes, speakers, parenthetical and dialogues have been extracted from the script

# ###  SCENE DATA IS READY, NOW CREATE THE WORD LIST ALONGWITH ITS ADDRESS TO WHICH IT BELONGS

new_scene=[]

scene_no=0
for scene in scenes:    #taking each scene
    if scene_no == 0:
        scene_no = scene_no+1
       # continue
    
    scene_dict={}
    action_counter=1
    dialogue_counter=1
    for line in scene:
        if type(line)==type(""):                       #taking each line in the scene
            if line.startswith('INT') or line.startswith('EXT') or line.startswith('EXT/INT') or line.startswith('INT/EXT'): #if it is a slug line
                scene_dict['SL'] = line
                slug_words=line.split(" ")             #extract words from each line

                for each_word in slug_words:           #tag each word
                    temp_word={}              
                    temp_word['word'] = each_word
                    temp_word['scene_num'] = scene_no
                    temp_word['type'] = 'SL'
                    temp_word['type_no'] = "" 
                    temp_word['sentence_no'] = '??'
                    temp_word['word_no_in sent'] = "??"
                    temp_word['POS'] = '??pos'
                    temp_word['importance'] = random_gen()
                    temp_word['zero_one'] = '1'
                    words.append(temp_word)             #creating word list
                    # print("temp word")
                    # print(temp_word)

            else:                 #if it is not a slug line
                scene_dict['AC'+ str(action_counter)] = line    #apparently an action line
                #print ("Scene_dic  ", scene_dict)
                sent_text = nltk.sent_tokenize(line)
                print("Sent_text", sent_text)
                sentence_no=0
                for sentence in sent_text:
                    tokens = nltk.word_tokenize(sentence)       #tokenize
                    postag=nltk.pos_tag(tokens)                 #calculate pos tags
                    #print ("posttag", postag)
                    sentence_no=sentence_no+1
                    word_no=0
                    for (token,pos) in postag:              
                        word_no=word_no+1
                        temp_word={}
                        temp_word['word'] = token
                        temp_word['scene_num'] = scene_no
                        temp_word['type'] = 'AC'             #action
                        temp_word['type_no'] = str(action_counter)
                        temp_word['sentence_no'] = sentence_no
                        temp_word['word_no_in_sent'] = word_no
                        temp_word['POS'] = pos
                        temp_word['importance'] = random_gen()
                        temp_word['zero_one'] = '1'
                        words.append(temp_word)              #along with the slug line words, action words are added
                        #print("temp word in action")
                    #print("Words in action", action_counter, )
                    #print(words)
                action_counter = action_counter + 1          #keep count of the actions

        else:                                               
            dialogue_words="dialoge"
            diag_list = line.keys()
            diag_list=list(diag_list)
            par_dia=line.values()
            par_dia=list(par_dia)
            f=diag_list[0]+"###"+par_dia[0][0]+"###"+par_dia[0][1]
            print(diag_list[0]+"###"+par_dia[0][0]+"###"+par_dia[0][1])
            scene_dict['DL'+ str(dialogue_counter)] = f   
            temp_word={}
            temp_word['word'] = diag_list[0]
            temp_word['scene_num'] = scene_no
            temp_word['type'] = 'DL_SPEAKER'
            temp_word['type_no'] = str(dialogue_counter)
            temp_word['sentence_no'] = '??'
            temp_word['word_no_in_sent'] = "??"
            temp_word['POS'] = 'NNP'
            temp_word['importance'] = random_gen()
            temp_word['zero_one'] = '1'
            words.append(temp_word)
            #print(temp_word) 
            parentheticals=par_dia[0][0].split(" ")
            word_no=0
            
            for each_word in parentheticals:          #analysing parentheticals
                if each_word=='NONE':
                    break
                word_no= word_no+1
                each_word = each_word.replace("(", "")
                each_word = each_word.replace(")", "")
                temp_word={}
                temp_word['word'] = each_word
                temp_word['scene_num'] = scene_no
                temp_word['type'] = 'DL_PARENTH'
                temp_word['type_no'] = str(dialogue_counter)
                temp_word['sentence_no'] = 1
                temp_word['word_no_in_sent'] = word_no
                temp_word['POS'] = '??pos'
                temp_word['importance'] = random_gen()
                temp_word['zero_one'] = '1'
                words.append(temp_word)
                #print(temp_word) 
            dialogues=par_dia[0][1].split(" ")        #analysing dialogues
            #print("dial=============", dialogues)
            sent_text = nltk.sent_tokenize(par_dia[0][1])
            sentence_no=0
            for sentence in sent_text:
                tokens = nltk.word_tokenize(sentence)
                postag=nltk.pos_tag(tokens)
                sentence_no=sentence_no+1
                word_no=0
                for (token,pos) in postag:
                    word_no=word_no+1
                    temp_word={}
                    temp_word['word'] = token
                    temp_word['scene_num'] = scene_no
                    temp_word['type'] = 'DL_DELIVERY'
                    temp_word['type_no'] = str(dialogue_counter)
                    temp_word['sentence_no'] = sentence_no
                    temp_word['word_no_in_sent'] = word_no
                    temp_word['POS'] = pos
                    temp_word['importance'] = random_gen()
                    temp_word['zero_one'] = '1'
                    words.append(temp_word)   
            dialogue_counter = dialogue_counter + 1
    ##see this
    scene_no= scene_no+1

    if scene_no > 1 : 
        new_scene.append(scene_dict)
print(" WORDS LIST\n")             #word list created
for word in words:
    print(word)
    print("\n")

"""
        a threshold value is calculated
        and the tokens whose importance is  < threshold value are removed


"""

def find_threshold(reduced_percent):
    """ Takes as imput the percentage of compression required in the script 
    and returns the 'threshold_val' - the thershold value of IMPORTANCE 
                the 'threshold_count' - the number of words in the reduced script
    """
    number_of_words=0 
    word_temp=[]
    for each_word in words:
        word_temp.append(each_word['importance'])
        number_of_words= number_of_words+1   
        #print (word_temp[number_of_words-1],number_of_words)   
    word_temp.sort() 
    reduced_percent=1-reduced_percent/100
    print("redu=",reduced_percent)
    threshold_count = round(number_of_words*reduced_percent)
    threshold_val = word_temp[threshold_count]
    print("threshold in fnc=", threshold_val, threshold_count)
    return threshold_val, threshold_count

def find_threshold_removal_impact(reduced_percent):
    """ Takes as imput the percentage of compression required in the script 
    and returns the 'threshold_val' - the thershold value of REMOVAL IMPACT 
                the 'threshold_count' - the number of words in the reduced script
    """
    number_of_words=0 
    word_temp=[]
    for each_word in words:
        word_temp.append(each_word['removal_impact'])
        number_of_words= number_of_words+1   
        #print (word_temp[number_of_words-1],number_of_words)   
    word_temp.sort() 
    reduced_percent=1-reduced_percent/100
    print("redu=",reduced_percent)
    threshold_count = round(number_of_words*reduced_percent)
    threshold_val = word_temp[threshold_count]
    print("threshold in fnc=", threshold_val, threshold_count)
    return threshold_val, threshold_count



def set_zero_initial(reduced_percent):
    """ Takes as input the percentage of reduced script and sets the words as '0' and '1' based on 
        the threshold value of importance
    """
    threshold_v, threshold_c = find_threshold(reduced_percent) 
    word_no=1
    for each_word in words:
        if each_word['importance'] < threshold_v:
            each_word['zero_one'] = '0'    
        else:
            each_word['zero_one'] = '1'
        word_importance.append(each_word['importance'])
        #print("importance zero=", each_word['zero_one'])
    return threshold_c



def set_zero_initial_removal_impact(reduced_percent):
    """ Takes as input the percentage of reduced script and sets the words as '0' and '1' based on 
        the threshold value of removal impact
    """
    threshold_v, threshold_c = find_threshold_removal_impact(reduced_percent) 
    word_no=1
    for each_word in words:
        #print("impact=",each_word['removal_impact'],"threshold=", threshold_v )
        if each_word['removal_impact'] <= threshold_v:
            each_word['zero_one'] = '0'   
        else:
            each_word['zero_one'] = '1'
        #print("impact zero=", each_word['zero_one'])
    return threshold_c

##########################     the priority list will store the index of the word with min. Word Importance

def convert_importance_to_priority(word_importance):
    """
    Takes the 'word_importance' as the input and define the 'priority' of the word for removal based on WORD IMPORTANCE
    e.g. if priority[1]=j   this means that the first priority for removal is of the  'jth WORD' 
            priority[5]=7   means 5th priority for removal is of the 7th word
            CAN BE MODIFIED USING SORTING OF REDUCED WORD IMPORTANCE
    """
    reduced_word_imp=list(word_importance)
    length = len(word_importance)
    minpos= word_importance.index(min(word_importance))
    k=0
    for i in range(length):
        priority.append(0)

    while(k<length):
        min_value = min(reduced_word_imp)
        minpos= word_importance.index(min_value)
        priority[k]=minpos
        reduced_word_imp.remove(min_value)
        k=k+1

def convert_removal_impact_to_priority(removal_impact_list):
    """
    Takes the 'removal impact list' as the input and define the 'priority' of the word for removal based on REMOVAL IMPACT VALUE OF THE WORD
    e.g. if priority[1]=j   this means that the first priority is of the  'jth WORD' (based on removal impact of word j)
            priority[5]=7   means that the 5th priority for removal is of the 7th word (based on removal impact of word 7)
    """
    unique_removal_impact_list = list(dict.fromkeys(removal_impact_list))
    unique_removal_impact_list.sort()
    length = len(removal_impact_list)
    priority_removal=[]
    for i in range(length):
        priority_removal.append(0)
    #print("small list",unique_removal_impact_list)
    priority_counter=0
    for each_unique in unique_removal_impact_list:
        word_no=0
        for each_removal_impact_value in removal_impact_list:
            if (each_unique == each_removal_impact_value):
                #print("uniq=",each_unique,  "all impact=",each_removal_impact_value)
                priority_removal[priority_counter] = word_no
                priority_counter=priority_counter+1
            word_no=word_no+1
    print("removal priority=",priority_removal)            
    return priority_removal


def removal_impact_sentence(word):
    """ Takes the word as input and calculates the removal impact, if the sentence it contains needs to be removed
        Values returned are
        'word_removal_impact' - sum of importance of all the words in the sentence that contains it
        'num_impacted'        - number of words in the sentence
        'impacted'            - list of word_no of the words in the the sentence
    """
    #print("word to check in sentence =",word['word'])
    comp_scene_no= word['scene_num']
    #print("comp=",comp_scene_no)
    comp_scene_type=word['type']
    comp_type_no=word['type_no']
    comp_sentence_no = word['sentence_no']
    word_no=0
    num_impacted=0
    impacted=[]
    word_removal_impact=0
    for each_word in words:
        #print("word_no=", word_no,"word is=", each_word)
        if(each_word['scene_num'] ==comp_scene_no and each_word['type'] == comp_scene_type and each_word['type_no'] == comp_type_no and each_word['sentence_no'] == comp_sentence_no):
            #print("word=", each_word['word'], "word_no=", word_no, "importance=",each_word['importance'] )
            word_removal_impact= word_removal_impact+ each_word['importance']
            impacted.append(word_no)
            num_impacted = num_impacted+1
        word_no=word_no+1
    #print("removal impact=", word_removal_impact)   
    #print("impacted list=", impacted) 
    return word_removal_impact, num_impacted, impacted      

def removal_impact_scene(word):
    """Takes the word as input and calculates the removal impact, if the SCENE it contains needs to be removed
        Values returned are
        'word_removal_impact' - sum of importance of all the words in the SCENE that contains it
        'num_impacted'        - number of words in the SCENE
        'impacted'            - list of word_no of the words in the SCENE
    """
    #print("removal of word in  scene=",word['word'])
    comp_scene_no= word['scene_num']
    #print("comp=",comp_scene_no)
    impacted=[]
    num_impacted=0
    word_no=0
    scene_removal_impact=0
    for each_word in words:
        if(each_word['scene_num'] == comp_scene_no):
            #print("Scene  word=", each_word['word'], "scene_num=", comp_scene_no, "scene_no=",each_word['scene_num'] )
            scene_removal_impact= scene_removal_impact+ each_word['importance']
            #print(each_word)
            impacted.append(word_no)
            #print("word_no=", word_no)
            num_impacted = num_impacted+1
        word_no=word_no+1
    #print("scene removal impact=", scene_removal_impact) 
    #print("impacted list=", impacted)  
    return scene_removal_impact, num_impacted, impacted 



def removal_impact_speaker(word):
    """Takes the word as input and calculates the removal impact, if the word is the SPEAKER of a dialogue
        Values returned are
        'word_removal_impact' - sum of importance of all the words including the corresponding DIALOGUE, PARENTHEICAL and SPEAKER
        'num_impacted'        - number of words impacted
        'impacted'            - list of word_no of the words that are impacted
    """
    comp_scene_no= word['scene_num']
    comp_scene_type=word['type']
    comp_type_no=word['type_no']
    num_impacted=0
    impacted=[]
    word_no=0
    speaker_removal_impact=0
    for each_word in words:
        if(each_word['scene_num'] ==comp_scene_no and each_word['type_no'] == comp_type_no):
            if (each_word['type'] == 'DL_DELIVERY' or each_word['type'] == 'DL_PARENTH' or each_word['type'] == 'DL_SPEAKER'):
               # print("Scene  word=", each_word['word'], "word_no=", word_no, "importance=",each_word['importance'] )
               # print("found", each_word['word'])
                speaker_removal_impact= speaker_removal_impact+ each_word['importance']
                impacted.append(word_no)
                num_impacted = num_impacted+1
        word_no=word_no+1
    #print("speaker removal impact=", speaker_removal_impact) 
    #print("impacted list=", impacted)   
    return speaker_removal_impact, num_impacted, impacted

def assign_word_removal_impact():
    """
    Calculates the removal impact of each word in the script 
        If the word is part of a slugline, the removal impact is sum of importance of all the words in the scene
        If the word is a speaker, the removal impact is sum of importance of all words in the dialoge, parenthetical and speaker
        If the word is a verb, the removal impact is sum of all words in the sentence that contains it
        DEFAULT REMOVAL IMPACT CHANGED FROM ZERO TO 1
    """
    removal_impact_value_list=[]
    for each_word in words:
        impacted_words=[]
        #print("  word=", each_word['word'])
        each_word['removal_impact'] = 1
        each_word['impacted_word_list'] = impacted_words

        if(each_word['type'] =='SL'):
            removal_impact, impacted_no, impacted_words=removal_impact_scene(each_word)
            each_word['removal_impact'] = removal_impact
            each_word['impacted_word_list'] = impacted_words

        if (each_word['type'] == 'DL_SPEAKER'):
            removal_impact, impacted_no, impacted_words=removal_impact_speaker(each_word)
            each_word['removal_impact'] = removal_impact
            each_word['impacted_word_list'] = impacted_words

        str=each_word['POS']
        #print (str, str[0:2])
        if (str[0:2] == 'VB'):
            removal_impact, impacted_no, impacted_words=removal_impact_sentence(each_word)
            each_word['removal_impact'] = removal_impact
            each_word['impacted_word_list'] = impacted_words
        
        #print ("removal impact =", each_word['removal_impact'])
        removal_impact_value_list.append(each_word['removal_impact'])
    return removal_impact_value_list


def calculate_a_by_b():
    """
    Calculate the theshold value of A/B that decides the stopping criteria
    """
    total_importance=0
    zero_importance=0
    for each_word in words:
        #print("each=", each_word)
        if each_word['zero_one'] == '0':
            zero_importance = zero_importance + each_word['importance']
        total_importance = total_importance + each_word['importance']  
    B= zero_importance/ total_importance      
    print("zero imp=", zero_importance, "total_imp=", total_importance, "B=",B)
    A = (100-reduced_percent)/100
    print("A= ", A)
    stop_threshold = A/B
    print("a_by_b_threshold=", stop_threshold)
    return stop_threshold

print ("\n  WORDS HAVE BEEN ASSIGEN IMPORTANCE AND ADDRESS")
    
reduced_percent=70
threshold_counter=set_zero_initial(reduced_percent)
convert_importance_to_priority(word_importance)
print("word importance=", word_importance)
print()
print("importance priority=",priority)
g_removal_impact_value_list=assign_word_removal_impact()
print()
print("removal impact values list=",g_removal_impact_value_list)
threshold_counter_removal_impact= set_zero_initial_removal_impact(reduced_percent)

r_priority=convert_removal_impact_to_priority(g_removal_impact_value_list)

a_by_b_threshold = calculate_a_by_b()


"""for each_word in words:
    #print("word=",each_word)
    print( "word=", each_word['word'], "impact=", each_word['removal_impact'], "zero_one=", each_word['zero_one'], )


for each_word in words:
    print("word=",each_word)
    print( "word=", each_word['word'], "importance=", each_word['importance'])
"""


Streaming output truncated to the last 5000 lines.

{'word': 'with', 'scene_num': 7, 'type': 'DL_DELIVERY', 'type_no': '25', 'sentence_no': 1, 'word_no_in_sent': 17, 'POS': 'IN', 'importance': 3123, 'zero_one': '1'}


{'word': 'nice', 'scene_num': 7, 'type': 'DL_DELIVERY', 'type_no': '25', 'sentence_no': 1, 'word_no_in_sent': 18, 'POS': 'JJ', 'importance': 1318, 'zero_one': '1'}


{'word': 'wine', 'scene_num': 7, 'type': 'DL_DELIVERY', 'type_no': '25', 'sentence_no': 1, 'word_no_in_sent': 19, 'POS': 'NN', 'importance': 1612, 'zero_one': '1'}


{'word': 'and', 'scene_num': 7, 'type': 'DL_DELIVERY', 'type_no': '25', 'sentence_no': 1, 'word_no_in_sent': 20, 'POS': 'CC', 'importance': 2804, 'zero_one': '1'}


{'word': 'tip', 'scene_num': 7, 'type': 'DL_DELIVERY', 'type_no': '25', 'sentence_no': 1, 'word_no_in_sent': 21, 'POS': 'NN', 'importance': 3072, 'zero_one': '1'}


{'word': '?', 'scene_num': 7, 'type': 'DL_DELIVERY', 'type_no': '25', 'sentence_no': 1, 'word_no_in_sent': 22, 'POS': '.

'for each_word in words:\n    #print("word=",each_word)\n    print( "word=", each_word[\'word\'], "impact=", each_word[\'removal_impact\'], "zero_one=", each_word[\'zero_one\'], )\n\n\nfor each_word in words:\n    print("word=",each_word)\n    print( "word=", each_word[\'word\'], "importance=", each_word[\'importance\'])\n'

In [0]:
"""
References: https://stanfordnlp.github.io/CoreNLP/

"""

['IT’S A MATCH!', 'EXT. MALL -EVENING', 'It’s a Saturday evening. The area around the mall is too\ncrowded. We see two guys coming out of the mall near the\ncorrider. One guy, Akash(35) is wearing shorts and a tshirt\nwith branded shoes carrying his gym bag and a sipper. Other\nguy, Kush(38) is wearing joggers with tshirt carrying a gym\nbag and a phone on the other hand. Their car "Mercedez S\nclass" arrives. As soon as they sit in the car Kush’s phone\nbuzzez.', 'INT. CAR. -EVENING', 'He gets a notification saying "Its a match". He\nunlocks his phone and gets excited by seeing the\nnotification. He opens the profile, he matched with Pooja.\nThere is a side picture of a woman with open straight hair\nand wearing big jhumkas. Kush’s not able to see the full\nface but just the side angle. He clicks on the profile to\nread the bio. The bio reads,', '"It doesn’t even matter whatever i write down here but still\nyou’re gonna swipe based on the pictures but still I am\nwriting.I am Delhite/

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
print(type(""))

<class 'str'>


In [0]:

filename="gdrive/My Drive/MNF_work/papers/itsamatch.txt"
characters = ['KUSH', 'POOJA', 'DRIVER', 'SHOPKEEPER', 'AKASH', 'RECEPTIONIST']
refined_line=[]

#word count
with open(filename, 'r') as f:   #open file 
    for line in f:
        words = line.split()
        NUM_WORDS_INFILE += len(words)   # count total no. of words
    print("Total number of words in the input file",NUM_WORDS_INFILE)

#read file
with open(filename, "r") as input:
    input_ = input.read().split('\n\n')

for line in input_:
    refined_line.append(line.strip())
print(refined_line)
